- [참조] https://wikidocs.net/55409

# `nn.Module`로 선형회귀(Linear Regression) 구현하기
- PyTorch에서 제공하는 함수 불러 사용 
  - 선형 회귀 모델: `nn.Linear()`
  - 평균 제곰 오차: `nn.functional.mse_loss()` 

## 1) 단순 선형 회귀 구현 
- `H(x) 식에 입력 값 `x`로 부터 예측된 `y`를 얻는 과정 => `forward 연산`
  - 학습 전: `prediction = model(x_train)` --> `x_train`으로부터 예측값 리턴 ==> `forward 연산` O
  - 학습 후:`pred_y = model(new_var)` --> 임의의 값 `new_var`로부터 예측값 리턴 ==> `forward 연산` O
- `forward 연산`: 학습 과정에서 비용함수를 미분하여 기울기(`W`) 구하는 것 
  - `cost.backward()`: 비용 함수로부터 기울기를 구하라는 의미

- 예시: `y = 2x`로 가정 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
torch.manual_seed(1)

In [4]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [5]:
# PyTorch에서 제공하는 선형 회귀 모델 
# model = nn.Linear(input_dim, output_dim)
# 모델을 선언 및 초기화. 단순 선형 회귀 --> 입력 차원: input_dim=1, 출력 차원: output_dim=1
model = nn.Linear(1,1)

In [7]:
# model -> 가중치(W), 편향(b) 저장됨
# model.parameters() 통해 불러올 수 있음 
# 리스트 첫번째가  W, 두번째가 b
# 두 값 모두 학습의 대상 --> requires_grad = True
print(list(model.parameters())) 

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


In [8]:
# optimizer 설정 
# 경사하강법 SGD 사용, learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [9]:
# 전체 훈련 데이터 2000번 경사하강법(SGD) 반복
nb_epochs = 2000
for epoch in range(nb_epochs+1):

  # 가설 H(x)
  prediction = model(x_train)

  # 비용함수 cost 계산 
  ### PyTorch에서 제공하는 평군 제곱 오차 함수 (MSE)
  cost = F.mse_loss(prediction, y_train)

  # cost -> H(x) 개선 
  optimizer.zero_grad() # gradient = 0 초기화
  cost.backward()       # cost 자동 미분 -> gradient 계산 
  optimizer.step()      # W, b 업데이트

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} | Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

## ====> 결과: cost의 값이 매우 작음 -> W, b 값도 최적화가 되었는지 확인하기

Epoch    0/2000 | Cost: 13.103541
Epoch  100/2000 | Cost: 0.002791
Epoch  200/2000 | Cost: 0.001724
Epoch  300/2000 | Cost: 0.001066
Epoch  400/2000 | Cost: 0.000658
Epoch  500/2000 | Cost: 0.000407
Epoch  600/2000 | Cost: 0.000251
Epoch  700/2000 | Cost: 0.000155
Epoch  800/2000 | Cost: 0.000096
Epoch  900/2000 | Cost: 0.000059
Epoch 1000/2000 | Cost: 0.000037
Epoch 1100/2000 | Cost: 0.000023
Epoch 1200/2000 | Cost: 0.000014
Epoch 1300/2000 | Cost: 0.000009
Epoch 1400/2000 | Cost: 0.000005
Epoch 1500/2000 | Cost: 0.000003
Epoch 1600/2000 | Cost: 0.000002
Epoch 1700/2000 | Cost: 0.000001
Epoch 1800/2000 | Cost: 0.000001
Epoch 1900/2000 | Cost: 0.000000
Epoch 2000/2000 | Cost: 0.000000


In [10]:
# 임의의값 x=4  대입 -> y 값 확인 
new_var  = torch.FloatTensor([[4.0]])

# 예측 y -> pred_y에 저장
pred_y = model(new_var)      # forward 연산 -> 자동 미분 

# y=2x --> 입력 x=4이면 y=8 가까운 값 나와야 제대로 학습 된 것
print(pred_y)

tensor([[7.9989]], grad_fn=<AddmmBackward0>)


In [11]:
# 학습 후 W, b 값 확인 
print(list(model.parameters()))

# # 학습 전: W = 0.5153, b=-0.4414
# [Parameter containing:
# tensor([[0.5153]], requires_grad=True), Parameter containing:
# tensor([-0.4414], requires_grad=True)]

[Parameter containing:
tensor([[1.9994]], requires_grad=True), Parameter containing:
tensor([0.0014], requires_grad=True)]
